# Running model with expression data

In [ ]:
%run util.py
from cobra.io import save_json_model, load_json_model
pubmod = MSModelUtil.from_cobrapy("data/TranslatedPublishedModel.json")
pubmod.model.reactions.get_by_id("rxn01332_c0").lower_bound = 0
pubmod.model.reactions.get_by_id("rxn01332_c0").upper_bound = 0
pyruvate_media = util.get_media("KBaseMedia/Carbon-Pyruvic-Acid")
growth_solution = util.run_fba(pubmod,media=pyruvate_media,objective="GROWTH_DASH_RXN",run_pfba=True)
#Forcing growth to be at least 50% of optimal
util.constrain_objective_to_fraction_of_optimum(pubmod, objective="GROWTH_DASH_RXN", fraction=0.5)
#Fitting model to expression data and knocking out reactions with low expression in each condition
expression = MSExpression.from_spreadsheet(filename="data/UGA_Proteomics_May2025_Report.xlsx",sheet_name="Imputed",skiprows=1,type="Log2",id_column="Protein Accession")
analysis = {}
condition_models = {}
for condition in expression.conditions:
    #Copying the model into conditions datastructure
    condition_models[condition.id] = MSModelUtil.from_cobrapy(cobra.io.json.to_json(pubmod.model))
    util.constrain_objective_to_fraction_of_optimum(condition_models[condition.id], objective="GROWTH_DASH_RXN", fraction=0.25)
    analysis[condition.id] = expression.fit_model_flux_to_data(
        model=condition_models[condition.id],
        condition=condition.id,
        default_coef=0.01,
        activation_threshold = None,  # cshenry 10/16/2026: Changed default for activation to None so activation will be off by default
        deactivation_threshold = 0.000001
    )
    analysis[condition.id].pop("solution")
    analysis[condition.id].pop("objective")
    #Setting reactions with low expression to zero flux
    for rxn in analysis[condition.id]["off_off"]:
        condition_models[condition.id].model.reactions.get_by_id(rxn).lower_bound = 0
        condition_models[condition.id].model.reactions.get_by_id(rxn).upper_bound = 0
#Fitting model to mutant growth rate data
mutant_growth_rates = MSExpression.from_spreadsheet(filename="data/MutantGrowthRatesData.xls",sheet_name="Mutant growth phenotypes",type="AbsoluteAbundance",id_column="Locus tag")
mutant_growth_rates.translate_data("NormalizedRatios")
analysis["MutantGrowth"] = expression.fit_model_flux_to_data(
    model=pubmod,
    condition="lactate",
    default_coef=0.01,
    activation_threshold = 0.1,  # cshenry 10/16/2026: Changed default for activation to None so activation will be off by default
    deactivation_threshold = 0.01
)
analysis["MutantGrowth"].pop("solution")
analysis["MutantGrowth"].pop("objective")

pubmod.model.objective = output["objective"]
rxn_fluxes = output["solution"].fluxes.to_dict()
util.create_map_html(model=pubmod.model,flux_solution=rxn_fluxes,map_name="Core",output_file="nboutput/escher_map.html")
util.save(analysis,"expression_analysis")

/Users/chenry/Dropbox/SyncCode/KBUtilLib/src
modelseedpy 0.4.2


2025-10-17 12:20:04,207 - __main__.NotebookUtil - WARNING - Section 'ModelSEEDBiochem' not found in config


cobrakbase 0.4.0


INFO:modelseedpy.core.msmodelutl:cpd00063 not found in model!
INFO:modelseedpy.core.msmodelutl:cpd10516 not found in model!
INFO:modelseedpy.core.msmodelutl:cpd00205 not found in model!
INFO:modelseedpy.core.msmodelutl:cpd00254 not found in model!
INFO:modelseedpy.core.msmodelutl:cpd00099 not found in model!
INFO:modelseedpy.core.msmodelutl:cpd00058 not found in model!
INFO:modelseedpy.core.msmodelutl:cpd00030 not found in model!
INFO:modelseedpy.core.msmodelutl:cpd00034 not found in model!
/opt/anaconda3/envs/modelseed/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)
/Users/chenry/Dropbox/SyncCode/KBUtilLib/src/kbutillib/dependencies/ModelSEEDpy/modelseedpy/multiomics/msexpression.py:742: RuntimeWarning: divide by zero encountered in scalar divide
  transformed_value = (2 ** (value - min_val)) / (2 ** (col_sum - n_features * min_val))
/Users/chenry/Dropbox/SyncCode/KBUtilLib/sr

KeyError: 'Protein Accession'